# Исследование по выбору начальной точки для оценки отклонения через виток

Посмотрим, как выбор начальной точки на орбите влияет на максимальное отклонение через период.

In [1]:
import numpy as np
from daceypy import array, DA
from utils.libration_sense import (initial_state_parser,
                                   CR3BP,
                                   RK78_trajectory,
                                   get_maxdeviation_initial_state,
                                   km2du,
                                   du2km,
                                   kmS2vu,
                                   get_maxdeviation_and_delta_from_initial_state,
                                   MU)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [13]:
orbit_type = 'L1'
orbit_num = 192

DA.init(3, 6)
x0, z0, vy0, T, _, __ = initial_state_parser(orbit_type, orbit_num)
initial_state = array.identity(6)
initial_state[0] += x0  # x
initial_state[2] += z0  # z
initial_state[4] += vy0  # v_y

number_of_turns = 1


trajectory_dots = RK78_trajectory(initial_state, 0.0, number_of_turns * T, CR3BP)

np.shape(trajectory_dots)

(66, 6)

In [14]:
np.random.seed(42)
std_pos = km2du(8)
std_vel = kmS2vu(0.05e-3)

In [15]:
init_dot = trajectory_dots[33]
init_state = array.identity(6)
init_state[0] += init_dot[0]  # x
init_state[1] += init_dot[1]  # y
init_state[2] += init_dot[2]  # z
init_state[3] += init_dot[3]  # vx
init_state[4] += init_dot[4]  # vy
init_state[5] += init_dot[5]  # vz

peri_bad_maxdev, peri_bad_delta_max, peri_bad_delta_min = get_maxdeviation_and_delta_from_initial_state(init_state,
                                                                                                        T,
                                                                                                        std_pos,
                                                                                                        std_vel,
                                                                                                        number_of_points=10000)
print(du2km(peri_bad_maxdev))
print(peri_bad_delta_max)
print(peri_bad_delta_min)

8675.493196733853
[ 2.91624180e-06  5.00832224e-05  5.32742236e-05 -1.89306886e-05
  7.22079630e-05 -5.41292864e-05]
[-8.77776021e-06 -2.80212987e-06 -3.46803674e-07 -1.71273292e-05
  1.15107766e-05  5.61966800e-05]


In [16]:
# Плохая перицентральная траектория
init_dot_bad = trajectory_dots[33]
init_state_bad = array.identity(6)
init_state_bad[0] += init_dot_bad[0] + peri_bad_delta_max[0]  # x
init_state_bad[1] += init_dot_bad[1] + peri_bad_delta_max[1]  # y
init_state_bad[2] += init_dot_bad[2] + peri_bad_delta_max[2] # z
init_state_bad[3] += init_dot_bad[3] + peri_bad_delta_max[3] # vx
init_state_bad[4] += init_dot_bad[4] + peri_bad_delta_max[4] # vy
init_state_bad[5] += init_dot_bad[5] + peri_bad_delta_max[5] # vz

trajectory_dots_bad = RK78_trajectory(init_state_bad, 0.0, number_of_turns * T, CR3BP)
trajectory_dots_bad.shape

(69, 6)

In [18]:
# Хорошая перицентральная траектория
init_dot_good = trajectory_dots[33]
init_state_good = array.identity(6)
init_state_good[0] += init_dot_good[0] + peri_bad_delta_min[0]  # x
init_state_good[1] += init_dot_good[1] + peri_bad_delta_min[1]  # y
init_state_good[2] += init_dot_good[2] + peri_bad_delta_min[2] # z
init_state_good[3] += init_dot_good[3] + peri_bad_delta_min[3] # vx
init_state_good[4] += init_dot_good[4] + peri_bad_delta_min[4] # vy
init_state_good[5] += init_dot_good[5] + peri_bad_delta_min[5] # vz

trajectory_dots_good = RK78_trajectory(init_state_good, 0.0, number_of_turns * T, CR3BP)
trajectory_dots_good.shape

(65, 6)

In [19]:
def plot_trajectory(dots: tuple[np.ndarray]) -> None:
    # Создаем фигуру и оси один раз перед циклом
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Добавляем Землю и Луну (один раз)
    earth_pos = np.array([-MU, 0, 0])
    moon_pos = np.array([1 - MU, 0, 0])
    ax.scatter(earth_pos[0], earth_pos[1], earth_pos[2], color='blue', s=200, label='Earth')
    ax.scatter(moon_pos[0], moon_pos[1], moon_pos[2], color='gray', s=50, label='Moon')
    
    for i, trajectory_dots in enumerate(dots):
        x = trajectory_dots[:, 0]
        y = trajectory_dots[:, 1]
        z = trajectory_dots[:, 2]
        
        # Рисуем траекторию с меткой
        ax.plot(x, y, z, label=f'Trajectory {i+1}', lw=1.5)
        ax.scatter(x, y, z, color='gray', s=10)
        
        # Добавляем номера точек (опционально)
        for j in range(len(x)):
            ax.text(x[j], y[j], z[j], str(j+1), fontsize=8)
    
    # Настраиваем оси и легенду один раз после цикла
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    ax.set_title('Trajectories graph')
    
    plt.show()

In [ ]:
%matplotlib qt
all_trajectories = [trajectory_dots_bad]
plot_trajectory(all_trajectories)

2025-04-12 16:07:08.127 Python[39811:15269874] _TIPropertyValueIsValid called with 16 on nil context!
2025-04-12 16:07:08.127 Python[39811:15269874] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-04-12 16:07:08.127 Python[39811:15269874] Text input context does not respond to _valueForTIProperty:


In [18]:
deviations_list = []
for init_dot in trajectory_dots:
    init_state = array.identity(6)
    init_state[0] += init_dot[0]  # x
    init_state[1] += init_dot[1]  # y
    init_state[2] += init_dot[2]  # z
    init_state[3] += init_dot[3]  # vx
    init_state[4] += init_dot[4]  # vy
    init_state[5] += init_dot[5]  # vz
    deviation =  get_maxdeviation_initial_state(init_state, T, std_pos, std_vel, number_of_points=10000)
    deviations_list.append(deviation)

KeyboardInterrupt: 

In [51]:
plt.scatter(range(len(deviations_list)), [du2km(dev) for dev in deviations_list])
plt.xlabel("Orbit point (initial state) [-]")
plt.ylabel("Max deviation [km]")
plt.grid(True)
plt.minorticks_on()
plt.show()

In [ ]:
%matplotlib qt
x = trajectory_dots[:, 0]
y = trajectory_dots[:, 1]
z = trajectory_dots[:, 2]

earth_pos = np.array([-MU, 0, 0])
moon_pos = np.array([1 - MU, 0, 0])

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, label='Trajectory', lw=1.5)
ax.scatter(x, y, z, color='gray', s=10)

for i in range(len(x)):
    ax.text(x[i], y[i], z[i], str(i+1), fontsize=8)

ax.scatter(earth_pos[0], earth_pos[1], earth_pos[2], color='blue', s=200, label='Earth')
ax.scatter(moon_pos[0], moon_pos[1], moon_pos[2], color='gray', s=50, label='Moon')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
ax.set_title('Initial Points graph')

plt.show()